In [1]:
# Imports

import polars as pl
import pandas as pd
import os
from datetime import datetime
from geopy.distance import geodesic

In [2]:
# Konstanter

CSV_EXTENSION = '.csv'
current_dir = os.getcwd()
relative_path_vin_data = '../LINX-data\\vin_kjøretøytype'
relative_path_positiondata = '../LINX-data\\posisjonsdata'

yearly_vin_data_filepaths = (
    os.path.join(current_dir, relative_path_vin_data, 'vin_kjøretøytype_2021' + CSV_EXTENSION),
    os.path.join(current_dir, relative_path_vin_data, 'vin_kjøretøytype_2022' + CSV_EXTENSION),
    os.path.join(current_dir, relative_path_vin_data, 'vin_kjøretøytype_2023' + CSV_EXTENSION),
    os.path.join(current_dir, relative_path_vin_data, 'vin_kjøretøytype_2024' + CSV_EXTENSION)
)

yearly_position_filepaths = (
    os.path.join(current_dir, relative_path_positiondata, '2021-01-01_2021-12-31' + CSV_EXTENSION),
    os.path.join(current_dir, relative_path_positiondata, '2022-01-01_2022-12-31' + CSV_EXTENSION),
    os.path.join(current_dir, relative_path_positiondata, '2023-01-01_2023-12-31' + CSV_EXTENSION),
    os.path.join(current_dir, relative_path_positiondata, '2024-01-01_2024-06-22' + CSV_EXTENSION)
)

yearly_vin_data_filepaths_gyro = (
    os.path.join(current_dir, relative_path_vin_data, 'vin_data_gyro', 'vin_kjøretøytype_2021' + CSV_EXTENSION),
    os.path.join(current_dir, relative_path_vin_data, 'vin_data_gyro', 'vin_kjøretøytype_2022' + CSV_EXTENSION),
    os.path.join(current_dir, relative_path_vin_data, 'vin_data_gyro', 'vin_kjøretøytype_2023' + CSV_EXTENSION),
    os.path.join(current_dir, relative_path_vin_data, 'vin_data_gyro', 'vin_kjøretøytype_2024' + CSV_EXTENSION)
)

yearly_position_filepaths_gyro = (
    os.path.join(current_dir, relative_path_positiondata, 'Posisjoner_SVV m gyro', '2021-01-01_2021-12-31' + CSV_EXTENSION),
    os.path.join(current_dir, relative_path_positiondata, 'Posisjoner_SVV m gyro','2022-01-01_2022-12-31' + CSV_EXTENSION),
    os.path.join(current_dir, relative_path_positiondata, 'Posisjoner_SVV m gyro','2023-01-01_2023-12-31' + CSV_EXTENSION),
    os.path.join(current_dir, relative_path_positiondata, 'Posisjoner_SVV m gyro','2024-01-01_2024-07-09' + CSV_EXTENSION)
)


In [3]:
# Proximity-funksjon

def calculate_proximity(df, truck_vins, trailer_vins, threshold=1):
    truck_trailer_pairs = {}

    for truck_vin in truck_vins:
        truck_data = df[df['VIN'] == truck_vin]
        
        for trailer_vin in trailer_vins:
            trailer_data = df[df['VIN'] == trailer_vin]
            
            distances = []
            
            for _, truck_row in truck_data.iterrows():
                trailer_row = trailer_data[trailer_data['Dato'] == truck_row['Dato']]
                
                if not trailer_row.empty:
                    truck_coords = (truck_row['Latitude'], truck_row['Longitude'])
                    trailer_coords = (trailer_row.iloc[0]['Latitude'], trailer_row.iloc[0]['Longitude'])
                    distance = geodesic(truck_coords, trailer_coords).kilometers
                    distances.append(distance)
            
            if distances and sum(distances) / len(distances) < threshold:
                if truck_vin not in truck_trailer_pairs:
                    truck_trailer_pairs[truck_vin] = []
                truck_trailer_pairs[truck_vin].append(trailer_vin)

    return truck_trailer_pairs


In [4]:
for i, year in enumerate([2021, 2022, 2023, 2024]):

    filepath_vin = yearly_vin_data_filepaths_gyro[i]
    filepath_position = yearly_position_filepaths_gyro[i]

    df_vin = pd.read_csv(filepath_vin)
    df_position = pd.read_csv(filepath_position)
    df_position['Dato'] = pd.to_datetime(df_position['Dato'])

    truck_vins = df_vin[df_vin['type'] == 'lastebil']['VIN'].unique()
    trailer_vins = df_vin[df_vin['type'] == 'tilhenger']['VIN'].unique()

    truck_trailer_pairs = calculate_proximity(df_position, truck_vins, trailer_vins)

    for truck_vin, trailers in truck_trailer_pairs.items():
        print(f"Lastebil {truck_vin} dro på tilhenger(e): {', '.join(trailers)} i {year}")

C:\Users\ander\AppData\Local\Temp\ipykernel_8828\3000394147.py:7: DtypeWarning: Columns (5,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_position = pd.read_csv(filepath_position)


Lastebil YV2R0P0G9JA834573 dro på tilhenger(e): YT9T2020213001426 i 2021
Lastebil YV2R0P0G9JA834573 dro på tilhenger(e): YT9T2020213001426 i 2022
Lastebil YV2RT60D3LA865845 dro på tilhenger(e): YS940100150HP1179 i 2022


C:\Users\ander\AppData\Local\Temp\ipykernel_8828\3000394147.py:7: DtypeWarning: Columns (15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df_position = pd.read_csv(filepath_position)


Lastebil YS2S6X40005544509 dro på tilhenger(e): YS950200160HP1228 i 2023
Lastebil YV2R0P0G9JA834573 dro på tilhenger(e): YT9T2020213001426 i 2023
Lastebil YS2R6X40002175084 dro på tilhenger(e): YS950200220HP1973 i 2023
Lastebil YS2S6X40002140896 dro på tilhenger(e): YT9T2022213001977 i 2023
Lastebil YS2S6X40002164298 dro på tilhenger(e): YT9T2021213001694 i 2023
Lastebil YV2RT40G8MA881846 dro på tilhenger(e): SZFPL0300ELD09098 i 2023
Lastebil YS2R6X40002174412 dro på tilhenger(e): YS950200190HP1555 i 2023
Lastebil YS2R6X40002186890 dro på tilhenger(e): YS950200150HP1176 i 2023
Lastebil YV2R0P0G2NA891249 dro på tilhenger(e): YT9T2018212001982, YT9T2015212001325 i 2023
Lastebil YS2S8X40005652227 dro på tilhenger(e): YT9T2021213001515 i 2023
Lastebil YV2R0P0G6NA310013 dro på tilhenger(e): YT9T2022213001726 i 2023


C:\Users\ander\AppData\Local\Temp\ipykernel_8828\3000394147.py:7: DtypeWarning: Columns (15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df_position = pd.read_csv(filepath_position)


Lastebil YV2R0P0D1KA852539 dro på tilhenger(e): YS940200230HP1957 i 2024
Lastebil YS2R6X40002170509 dro på tilhenger(e): YT9T2018212001983 i 2024
Lastebil YS2R6X40002175084 dro på tilhenger(e): YS950200220HP1973 i 2024
Lastebil YS2R6X40005583741 dro på tilhenger(e): YT9T2015212001217 i 2024
Lastebil YS2R6X40002147130 dro på tilhenger(e): YT9T2014212001092 i 2024
Lastebil YS2S6X40002164298 dro på tilhenger(e): YT9T2021213001694 i 2024
Lastebil YV2RT40G8MA881846 dro på tilhenger(e): SZFPL0300ELD09098 i 2024
Lastebil YS2R6X40002170595 dro på tilhenger(e): YS940200190HP1512 i 2024
Lastebil YS2R6X40002158609 dro på tilhenger(e): YS950200210HP1810 i 2024
Lastebil YS2R6X40002186890 dro på tilhenger(e): YT9T2014212001092 i 2024
Lastebil YV2R0P0G2NA891249 dro på tilhenger(e): YT9T2018212001982, YT9T2015212001325 i 2024
Lastebil YS2S8X40005652227 dro på tilhenger(e): YT9T2021213001515 i 2024
Lastebil YV2R0P0G6NA310013 dro på tilhenger(e): YT9T2022213001726 i 2024
